In [10]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################


username = "aacuser"
password = "password_1"


# Connect to database via CRUD Module
db = AnimalShelter()

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)

#Dictionary for retrieving data
rescue_dict = {
    'Water Rescue': {
        "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
        "sex_upon_outcome": "Intact Female",
        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
    },
    'Mountain or Wilderness Rescue': {
        "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog",
                          "Siberian Husky", "Rottweiler"]},
        "sex_upon_outcome": "Intact Male",
        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
    },
    'Disaster or Individual Tracking': {
        "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever",
                          "Bloodhound", "Rottweiler"]},
        "sex_upon_outcome": "Intact Male",
        "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
    },
    'Reset': {}
}



#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()



# Layout definition
app.layout = html.Div([
    # Logo and Header
    html.Div([
        html.A(
            html.Img(
                src='data:image/png;base64,{}'.format(encoded_image),
                style={'height': '100px'}
            ),
            href='https://www.snhu.edu',
            target='_blank'
        ),
        html.H1("Grazioso Salvare Rescue Dashboard"),
        html.H4("Created by Darian Gernand")
    ], style={'textAlign': 'center'}),
    
    html.Hr(),
    # Dropdown for filtering rescue type
    html.Div([
        html.Label("Select Rescue Type:"),
        dcc.Dropdown(
            id='filter-type',
            options=[{'label': k, 'value': k} for k in rescue_dict.keys()],
            value='Reset',
            style={'width': '50%'}
        )
    ], style={'padding': '10px'}),

    html.Hr(),

    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        filter_action='native',
        sort_action='native',
        row_selectable='single',
        selected_rows=[0],
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left'},
        style_header={'backgroundColor': 'rgb(210, 210, 210)', 'fontWeight': 'bold'}
    ),

    html.Hr(),

    # Graph and Map
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_value):
    query = rescue_dict.get(filter_value, {})
    results = db.read(query)
    df = pd.DataFrame.from_records(results)
    df.drop(columns=['_id'], inplace=True)
    return df.to_dict('records')

# Update chart when table is filtered
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    if viewData is None or len(viewData) == 0:
        return html.Div("No data to display.")

    dff = pd.DataFrame.from_dict(viewData)

    # Group by breed and count occurrences
    df_grouped = dff['breed'].value_counts().reset_index()
    df_grouped.columns = ['breed', 'count']

    # Create pie chart
    fig = px.pie(df_grouped, values='count', names='breed', title="Animal Breeds")

    # Increase chart size
    fig.update_layout(
        height=600,
        width=600,
        margin=dict(t=40, b=40, l=40, r=40)
    )

    return [dcc.Graph(figure=fig)]

# Highlight selected columns
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    Input('datatable-id', 'selected_columns')
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns or []]

# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if not viewData or index is None or len(index) == 0:
        return html.Div("No animal selected.")

    dff = pd.DataFrame.from_dict(viewData)

    # Get the row based on selected index
    selected_row = dff.iloc[index[0]]

    # Use column names to pull data
    lat = selected_row.get('location_lat')
    lon = selected_row.get('location_long')
    breed = selected_row.get('breed', 'Unknown Breed')
    name = selected_row.get('name', 'Unknown')

    # Validate coordinates exist
    if pd.isnull(lat) or pd.isnull(lon):
        return html.Div("No valid location data for selected animal.")

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[lat, lon], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(breed),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(name)
                ])
            ])
        ])
    ]



app.run_server(debug=True)


Dash app running on http://127.0.0.1:17686/
Total records returned: 39940
DataFrame shape: (39940, 16)
Duplicate rows: 29955
Total records returned: 20
DataFrame shape: (20, 16)
Duplicate rows: 15


KeyError: "['_id'] not found in axis"